In [73]:
import pandas as pd
import requests
import csv
import os
import sys
import time
from collections import defaultdict

csv.field_size_limit(sys.maxsize)

username = ''
token = ''


In [86]:
file_path = 'others_projects_Java_list.txt'  # list of all Apache projects

# Open the file in read mode
with open(file_path, 'r') as file:
    for line in file:  
        line = line.strip() # Access each project line by line file reading            
        
        loc = line.find('_')
        pro = line[:loc]
        rep = line[loc+1:]        
        
        # prepare the path for accessing specific project information
        #project_name = "apache_"+str(line)
        project_name = str(line)
        #project_fullpath = "/u1/mdr614/Data_for_files_ordering_research/Apache_projects_java/" + project_name +"/"+project_name
        project_fullpath = "/u1/mdr614/Data_for_files_ordering_research/Others/" + project_name +"/"+project_name
        #print(project_fullpath)
        
        # Load the CSV file
        file_name = project_fullpath+"_basicCommentInfo.csv"
        print('FileName: ',file_name)
        
        
        
        df = pd.read_csv(file_name)
        #print(df.head())
        
        filtered_df = df[df['state'] == 'CHANGES_REQUESTED']  # only the rows where reviewers requested for code changes

        # Save the filtered rows to a new CSV file
        new_filename = project_name+"_changeRequested.csv"
        filtered_df.to_csv(new_filename, index=False)

        #print("Filtered rows have been saved to "+ new_filename)
        
        df = pd.read_csv(new_filename)  # open the file to extract the pull request number for API request 
        #df.head()
        pull_has_changeRequest = df['pullNumber'].tolist()
        pull_has_changeRequest = sorted(set(pull_has_changeRequest))  # get the unique pull number and sort in accending order

        # Print the list
        #print(pull_has_changeRequest) 
        print('Length: ',len(pull_has_changeRequest)) # total number of pull has change requested

        # for each pull request number, apply github API to get diff hunk and comments
        for i in range(len(pull_has_changeRequest)):
            
            if int(pull_has_changeRequest[i]) < 19243 and line == "":
                print(pull_has_changeRequest[i])
                continue

            # Define the GitHub API URL for the specific project (e.g., pull request data)
            #api_url = 'https://api.github.com/repos/apache/'+line+'/pulls/'+str(pull_has_changeRequest[i])+'/comments'
            api_url = 'https://api.github.com/repos/'+pro+"/"+rep+'/pulls/'+str(pull_has_changeRequest[i])+'/comments'
            #api_url = 'https://api.github.com/repos/apache/'+line+'/pulls/'+str(13137)+'/comments'
            
            print(api_url)
            #time.sleep(1.8)

            # Make the API request
            response = requests.get(api_url,auth=(username,token))
            data_json = response.json()  #get the json response in a variable
            #print(type(data_json))
            #print(len(data_json))
            

            if len(data_json)==0 or isinstance(data_json, dict):  #if REST api does not return anything. Means no pull left
                print("Json file empty!")

            else: 

                diff_hunk = []   # diff-hunk that is targeted for code change comments
                comment = []     # change comment and other discussion by reviewers and author
                commenter = []   # Nature of contributor (Reviewer or Author)
                pulls = []       # pull request number
                paths = []       # Name of the filepath that carries the diff-hunk

                for p in range(len(data_json)):  #if REST api returns pull requests information
                    d = data_json[p] 
                    #print(d)

                    diff_hunk.append(d['diff_hunk'])
                    comment.append(d['body'])
                    if d['user']is not None:
                        commenter.append(d['user']['login'])
                    else:
                        commenter.append("Unknown")

                    pulls.append(pull_has_changeRequest[i])
                    paths.append(d['path'])


                # Dictionary to store comments and commenters grouped by diff_hunk
                grouped_data = defaultdict(list)

                # Iterate over the variables and group the comments and commenters by d_hunk
                for hunk, comm, commenter in zip(diff_hunk, comment, commenter):
                    grouped_data[hunk].append({'comment': comm, 'commenter': commenter})

                # Convert the grouped data into a list of dictionaries (or DataFrame if needed)
                grouped_list = [{'d_hunk': hunk, 'comments': comments} for hunk, comments in grouped_data.items()]

                #print(grouped_list)

                # Convert to DataFrame
                df = pd.DataFrame(grouped_list)

                # Save to CSV (if needed)
                df.to_csv('grouped_comments.csv', mode='a', header=False, index=False)  # this file has diff-hunk and associate all comments

                # Create a DataFrame from the lists
                df = pd.DataFrame({         # this datafram is for gather all the together 
                    'Column1': pulls,
                    'Column2': paths,
                    'COlumn3' : diff_hunk
                })

                # Save the DataFrame to a CSV file
                df.to_csv('output_file.csv', mode='a', header=False, index=False)  # this file has pull number, paths and diff-hunk

                # Print the DataFrame to verify
                #print(df)
                
                # Step 1: Load the CSV file without headers
                input_file = 'output_file.csv'  
                output_file = 'filtered_rows.csv'   # Output file to save the unique rows based on diff-hunk

                # Set to store unique values in the third column
                unique_third_column_values = set()

                # List to store rows that need to be saved
                filtered_rows = []

                #Read the CSV file and filter rows based on the third column
                with open(input_file, mode='r', newline='') as csvfile:
                    reader = csv.reader(csvfile)


                    for row in reader:
                        if len(row) >= 3:  # Ensure there are at least 3 columns in the row
                            third_column_value = row[2]
                            if third_column_value not in unique_third_column_values:
                                unique_third_column_values.add(third_column_value)
                                filtered_rows.append(row)
                            else:
                                # If the value in the third column is already encountered, skip adding this row
                                continue

                # Step 2: Write the filtered rows to a new CSV file
                with open(output_file, mode='w', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerows(filtered_rows)

                print(f"Filtered rows have been saved to '{output_file}'.")
                
        # following code merge the two CSVs to create a new one with pull, path, diff-hunk and comments 
        first_csv = 'grouped_comments.csv'  # Replace with your first CSV file path
        second_csv = 'filtered_rows.csv'  # Replace with your second CSV file path
        output_csv = project_name+'_ChangeRequestComments.csv'  # Output file to save the merged rows

        # Custom headers for the merged CSV file
        custom_headers = ['Pull', 'Path', 'Diff_hunk', 'Comment']  # Replace with your desired headers

        # Dictionary to store rows from the first CSV based on the first column
        first_csv_data = {}

        # Read the first CSV and store its data
        with open(first_csv, mode='r', newline='') as file1:
            reader1 = csv.reader(file1)

            for row in reader1:
                if len(row) > 0:  # Ensure there is at least one column
                    first_csv_data[row[0]] = row  # Use the first column as the key

        # Open the second CSV and merge the rows based on the common column
        with open(second_csv, mode='r', newline='') as file2, open(output_csv, mode='w', newline='') as outfile:
            reader2 = csv.reader(file2)
            writer = csv.writer(outfile)

            # Write the custom headers to the output CSV
            writer.writerow(custom_headers)

            for row in reader2:
                if len(row) >= 3:  # Ensure there are at least 3 columns in the row
                    common_value = row[2]
                    if common_value in first_csv_data:
                        # Merge the rows, but remove the common column from the first CSV data
                        merged_row = row + first_csv_data[common_value][1:]  # Skip the first column from the first CSV data
                        writer.writerow(merged_row)

        print(f"Merged CSV file has been saved to '{output_csv}'.")
        
        
        

        
        
        # delete the intermediary CSVs
        csv_files_to_delete = [
            'filtered_rows.csv',  # Replace with the path to your CSV file
            'grouped_comments.csv',  # Add more file paths as needed
            'output_file.csv'
        ]

        # Loop through the list and delete each file
        for file in csv_files_to_delete:
            try:
                os.remove(file)
                print(f"Deleted {file} successfully.")
            except FileNotFoundError:
                print(f"{file} not found.")
            except PermissionError:
                print(f"Permission denied: {file}.")
            except Exception as e:
                print(f"Error deleting {file}: {e}")
                
        
        print('Done:', line)
        print()
        print()
        





FileName:  /u1/mdr614/Data_for_files_ordering_research/Others/jumpserver_jumpserver/jumpserver_jumpserver_basicCommentInfo.csv
Length:  8
https://api.github.com/repos/jumpserver/jumpserver/pulls/2547/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/jumpserver/jumpserver/pulls/2571/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/jumpserver/jumpserver/pulls/3698/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/jumpserver/jumpserver/pulls/6236/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/jumpserver/jumpserver/pulls/8187/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/jumpserver/jumpserver/pulls/8188/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/jumpserver/jumpserver/pulls/8648/comments
Filtered rows have been saved to 'filtere

Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/876/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/883/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/891/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/898/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/917/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/941/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/952/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/995/comments


Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/1729/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/1745/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/1753/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/1756/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/1773/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/1791/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/1799/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/1811/c

Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/2847/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/2851/comments
Json file empty!
https://api.github.com/repos/lightning-ai/lightning/pulls/2852/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/2853/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/2865/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/2866/comments
Json file empty!
https://api.github.com/repos/lightning-ai/lightning/pulls/2874/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/2892/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.

Json file empty!
https://api.github.com/repos/lightning-ai/lightning/pulls/4250/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/4256/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/4259/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/4273/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/4278/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/4282/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/4301/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/lightning-ai/lightning/pulls/4309/comments
Filtered rows have been saved

...

Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/8755/comments
Json file empty!
https://api.github.com/repos/serverless/serverless/pulls/8756/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/8765/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/8766/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/8768/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/8778/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/8780/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/8784/comments
Filtered rows have been saved to 'fil

Json file empty!
https://api.github.com/repos/serverless/serverless/pulls/9267/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9270/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9296/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9301/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9304/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9308/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9313/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9314/comments
Filtered rows have been saved to 'fil

Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9884/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9892/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9895/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9903/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9904/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9909/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9912/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/9914/comments


Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/11272/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/11298/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/11345/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/11367/comments
Json file empty!
https://api.github.com/repos/serverless/serverless/pulls/11460/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/11461/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/11479/comments
Filtered rows have been saved to 'filtered_rows.csv'.
https://api.github.com/repos/serverless/serverless/pulls/11506/comments
Json file empty!
https://api.

FileNotFoundError: [Errno 2] No such file or directory: '/u1/mdr614/Data_for_files_ordering_research/Others//_basicCommentInfo.csv'